In [9]:
import os
from dotenv import load_dotenv

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import numpy as np
import math
from tqdm import tqdm

In [2]:
load_dotenv()
ID = os.getenv("ID")
SECRET = os.getenv("SECRET")

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=ID, client_secret=SECRET))

In [3]:
df = pd.read_csv("tracks_with_uri.csv")
df["duration_ms"] = 0
df.set_index("master_metadata_track_name", inplace=True)
df

,spotify_track_uri,duration_ms
master_metadata_track_name,,
"""Das ist ein Freimaurer-Raum?"" fragte Sato",spotify:track:3IISUM22gI0PTJU5tpwfaZ,0
"""Professor Langdon?"" fragte Sato",spotify:track:00Kgmmb3sLa98LmXLx7Sae,0
"""The Take Over, The Breaks Over""",spotify:track:26HZ4xullUsdyMx8QnCbmO,0
"""Warum antwortet Peter nicht?""",spotify:track:1TFvampZQpx0LLxIEFDbh1,0
"""Wohin bringen sie mich?"" dachte Bellamy",spotify:track:7FKOK3DBtkNLfZkqNBb4Zc,0
...,...,...
早い話,spotify:track:59VqqJdqGIP1h14I797eO0,0
真夜中のオーケストラ,spotify:track:4ARcfREq6OpIVfVImAPFpc,0
青と夏,spotify:track:5BC6kr6etk2Y9J62AyI4i3,0


In [4]:
test_track = df.iloc[0]
test_track

spotify_track_uri    spotify:track:3IISUM22gI0PTJU5tpwfaZ
duration_ms                                             0
Name: "Das ist ein Freimaurer-Raum?" fragte Sato, dtype: object

In [5]:
uris = np.column_stack([df.index.tolist(), df["spotify_track_uri"].tolist()])
uris = df["spotify_track_uri"].tolist()
uris[0]

'spotify:track:3IISUM22gI0PTJU5tpwfaZ'

<div class="alert alert-block alert-info">
    At the moment, not all result can be retraced to a specific track from the list.
    Requesting them individually seems not possible. <br>
    The immediate fix would be to get as many tracks done and solve the remaining by hand.
</div>

In [ ]:
for chunk in tqdm(np.array_split(uris,math.ceil(len(uris)/50))):
    result = sp.tracks(chunk, market="DE")
    for song_info in result["tracks"]:
        try:
            df.loc[song_info["name"], "duration_ms"] = song_info["duration_ms"]
        except:
            continue
df

In [47]:
genres = []
for chunk in tqdm(np.array_split(uris,math.ceil(len(uris)/20))):
    result = sp.tracks(chunk, market="DE")
    artists = []
    for song_info in result["tracks"]:
        for artist in song_info["artists"]:
            artists.append(artist["id"])
    artists_result = sp.artists(artists)
    for artist in artists_result["artists"]:
        if not any(a["id"] == artist["id"] for a in genres):
            genres.append({"name": artist["name"], "id": artist["id"], "genres": artist["genres"]})
df_artists = pd.DataFrame(genres)
df_artists
#sp.artist(sp.track(uris[5], market="DE")["artists"][0]["id"])["genres"]

  6%|██▌                                       | 53/881 [00:32<08:35,  1.61it/s]


ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)